# SETUP

In [5]:
!pip uninstall -y numpy protobuf tensorflow keras opencv-python opencv-contrib-python opencv-python-headless jax ml-dtypes
!pip install numpy==2.1.2 protobuf==5.29.1 tensorflow==2.19.0 keras==3.5.0 ml-dtypes==0.5.0 opencv-python opencv-contrib-python


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: protobuf 4.25.8
Uninstalling protobuf-4.25.8:
  Successfully uninstalled protobuf-4.25.8
Found existing installation: tensorflow 2.14.1
Uninstalling tensorflow-2.14.1:
  Successfully uninstalled tensorflow-2.14.1
Found existing installation: keras 2.14.0
Uninstalling keras-2.14.0:
  Successfully uninstalled keras-2.14.0
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
Found existing installation: jax 0.5.3
Uninstalling jax-0.5.3:


In [1]:
# Install & imports (run this first in Colab)
!pip install -q tensorflow==2.14.1 scikit-learn matplotlib seaborn umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python-he

In [1]:
# Colab-ready script: InceptionResNetV2 (Frozen / Finetune) vs Triplet Siamese
# Save this as a .py file or paste into a Colab cell. Run cell-by-cell in Colab.
# Paths: change DATA_DIR to your dataset root (folders per identity)
# Models and results will be saved to: /content/drive/MyDrive/Kodingan/hasil/

import os
import random
import math
import time
from glob import glob
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import image_dataset_from_directory

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE
from sklearn.utils import resample

In [3]:
!unzip "/content/drive/MyDrive/Kodingan/dataset_processed.zip" -d "/content"
!unzip "/content/drive/MyDrive/Kodingan/dataset_testing_final.zip" -d "/content"
!unzip "/content/drive/MyDrive/Kodingan/best_testing.zip" -d "/content"

Archive:  /content/drive/MyDrive/Kodingan/dataset_processed.zip
  inflating: /content/dataset_processed/class_19/19S3INA1.jpg  
  inflating: /content/dataset_processed/class_19/19S2IIR1.jpg  
  inflating: /content/dataset_processed/class_31/31S3IIR1.jpg  
  inflating: /content/dataset_processed/class_26/26S2IAR1.jpg  
  inflating: /content/dataset_processed/class_28/28S2IIR1.jpg  
  inflating: /content/dataset_processed/class_38/38S2INA1.jpg  
  inflating: /content/dataset_processed/class_38/38S4IIR1.jpg  
  inflating: /content/dataset_processed/class_28/28S4IIR1.jpg  
  inflating: /content/dataset_processed/class_33/33S2IIR1.jpg  
  inflating: /content/dataset_processed/class_26/26S3IAR1.jpg  
  inflating: /content/dataset_processed/class_38/38S2IIR1.jpg  
  inflating: /content/dataset_processed/class_21/21S4IAR1.jpg  
  inflating: /content/dataset_processed/class_41/41S2IIR1.jpg  
  inflating: /content/dataset_processed/class_38/38S4IAR1.jpg  
  inflating: /content/dataset_processed/

# CONFIG

In [7]:
# %%
# CONFIG
DATA_DIR = '/content/dataset_processed'  # <-- change if needed
OUTPUT_DIR = '/content/drive/MyDrive/Kodingan/hasil'
IMG_SIZE = (299, 299)
BATCH_SIZE = 32
EMBED_DIM = 512
SEEDS = [42, 7, 123, 2023, 999]  # seeds for repeated experiments
GALLERY_K = 8  # number of gallery images per identity
EPOCHS = 30

MAIN_CLASS_NAMES = [
    "Wahyu", "Syahid", "Raiz", "Nurhatinah", "Mahdaniar",
    "Iska", "Erwin", "Bulan", "Amri"
]

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
# %%
# Utilities: list identities, image files
def list_identities(data_dir):
    classes = [d for d in sorted(os.listdir(data_dir)) if os.path.isdir(os.path.join(data_dir, d))]
    return classes

classes = list_identities(DATA_DIR)
print(f'Found {len(classes)} identities')

Found 50 identities


# DATASET

In [ ]:
# %%
# Build gallery/probe splits reproducibly per seed
def make_splits(data_dir, gallery_k=GALLERY_K, seed=42):
    rng = random.Random(seed)
    gallery = {}
    probe = {}
    for cls in sorted(os.listdir(data_dir)):
        cls_dir = os.path.join(data_dir, cls)
        images = sorted([os.path.join(cls_dir, f) for f in os.listdir(cls_dir)
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        if len(images) <= gallery_k:
            # if not enough images, use all-1 as probe
            g = images[:-1]
            p = images[-1:]
        else:
            rng.shuffle(images)
            g = images[:gallery_k]
            p = images[gallery_k:]
        gallery[cls] = g
        probe[cls] = p
    return gallery, probe

# quick check
g,p = make_splits(DATA_DIR, gallery_k=GALLERY_K, seed=42)
print('Per-class gallery size examples:', {k: len(v) for k,v in list(g.items())[:3]})

In [ ]:
# %%
# Dataset pipeline helpers
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_image(path):
    # read, decode, resize, scale
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_dataset_from_filepaths(filepaths, labels=None, batch_size=BATCH_SIZE, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(filepaths)
    ds = ds.map(lambda x: preprocess_image(x), num_parallel_calls=AUTOTUNE)
    if labels is not None:
        ds = tf.data.Dataset.zip((ds, tf.data.Dataset.from_tensor_slices(labels)))
    if shuffle:
        ds = ds.shuffle(1024)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

# MODEL INIT

In [ ]:
# %%
# Build model constructors

def build_backbone(input_shape=(*IMG_SIZE,3)):
    base = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    x = base.output
    x = layers.GlobalAveragePooling2D(name='AvgPool')(x)
    x = layers.Dropout(0.2, name='Dropout')(x)
    x = layers.Dense(EMBED_DIM, use_bias=False, name='Bottleneck')(x)
    x = layers.BatchNormalization(momentum=0.995, epsilon=0.001, scale=False, name='Bottleneck_BatchNorm')(x)
    embedding = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1), name='Embedding')(x)
    model = tf.keras.Model(inputs=base.input, outputs=embedding)
    return model, base

# For classification head we will append a Dense(num_classes, activation='softmax')
def build_classifier_model(num_classes, backbone, freeze_backbone=True, unfreeze_last_n=0):
    # backbone is returned as base model inside build_backbone
    base = backbone
    if freeze_backbone:
        for layer in base.layers:
            layer.trainable = False
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.trainable = False
    # optionally unfreeze last n layers (except BN)
    if unfreeze_last_n > 0:
        for layer in base.layers[-unfreeze_last_n:]:
            if not isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.trainable = True

    inputs = base.input
    x = base.output
    # classification head
    logits = layers.Dense(num_classes, activation='softmax', name='Logits')(x)
    model = tf.keras.Model(inputs=inputs, outputs=logits)
    return model

# TRAINING FUNCTION

In [ ]:
# %%
# Losses & training loops

# Triplet batch-hard mining utilities

def pairwise_distances_np(x):
    # x: [N, D]
    return pairwise_distances(x, metric='euclidean')

def batch_hard_triplet_loss(embeddings, labels, margin=0.2):
    # embeddings: numpy array [N,D], labels: array [N]
    distances = pairwise_distances_np(embeddings)
    N = distances.shape[0]
    loss = 0.0
    triplets = 0
    for i in range(N):
        label_i = labels[i]
        pos_mask = (labels == label_i)
        neg_mask = (labels != label_i)
        pos_mask[i] = False
        if pos_mask.sum() == 0:
            continue
        hardest_pos = distances[i][pos_mask].max()
        hardest_neg = distances[i][neg_mask].min()
        triplet_loss = max(0.0, margin + hardest_pos - hardest_neg)
        loss += triplet_loss
        triplets += 1
    return loss / max(1, triplets)

# Keras training loop for classifier

def train_classifier(model, train_ds, val_ds, outdir, epochs=EPOCHS, lr=1e-4):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(os.path.join(outdir, 'classifier_best.h5'), save_best_only=True, monitor='val_accuracy', mode='max'),
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
    ]
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)
    return history

# Triplet training: we will implement an epoch-based loop that constructs batches with P x K

def prepare_pk_batch(filepaths, labels, P=8, K=4):
    # labels are class strings; we will sample P identities and K images each
    unique = sorted(list(set(labels)))
    chosen = random.sample(unique, min(P, len(unique)))
    batch_files = []
    batch_labels = []
    for c in chosen:
        idxs = [i for i,l in enumerate(labels) if l==c]
        picks = random.choices(idxs, k=K) if len(idxs) < K else random.sample(idxs, k=K)
        for pi in picks:
            batch_files.append(filepaths[pi])
            batch_labels.append(c)
    return batch_files, batch_labels


def train_triplet(backbone_model, filepaths, labels, val_filepaths, val_labels, outdir,
                  epochs=EPOCHS, P=8, K=4, lr=1e-4, margin=0.2):
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    steps_per_epoch = max(100, len(filepaths)//(P*K))
    val_embeddings = None
    for epoch in range(epochs):
        t0 = time.time()
        losses = []
        for step in range(steps_per_epoch):
            batch_files, batch_labels = prepare_pk_batch(filepaths, labels, P=P, K=K)
            imgs = np.stack([preprocess_image(p).numpy() for p in batch_files], axis=0)
            labels_arr = np.array(batch_labels)
            with tf.GradientTape() as tape:
                emb = backbone_model(imgs, training=True)
                emb_np = emb.numpy()
                loss = batch_hard_triplet_loss(emb_np, labels_arr, margin=margin)
            # compute gradients w.r.t trainable variables using surrogate: compute a scalar loss via tf.constant
            # This is approximate: we'll apply gradients from L2 norm of embeddings differences weighted by triplet structure.
            # Simpler approach: use a proxy loss that is differentiable via tf functions.
            # Implementing full batch-hard in pure TF is long; we'll instead use semi-hard triplet via TF API for stability.
            # Fallback: break to use tf.keras.losses.TripletSemiHardLoss
            pass
        # Using TF native TripletSemiHardLoss for actual training loop (simpler and stable)
        break

    # Simpler robust implementation: use a dataset and compile with TripletSemiHardLoss
    # Rebuild a Keras model that outputs embeddings and train with a dataset of (image, label)
    labels_int, label_to_int = {}, {}
    unique = sorted(list(set(labels)))
    for i,c in enumerate(unique):
        label_to_int[c] = i
    labels_int = np.array([label_to_int[l] for l in labels])

    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels_int))
    def _map(p,l):
        return preprocess_image(p), l
    ds = ds.map(lambda p,l: _map(p,l), num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(1024).batch(P*K).prefetch(AUTOTUNE)

    # compile with TripletSemiHardLoss
    backbone_model.compile(optimizer=optimizer, loss=tf.keras.losses.TripletSemiHardLoss())
    callbacks = [tf.keras.callbacks.ModelCheckpoint(os.path.join(outdir, 'triplet_best.h5'), save_best_only=True)]
    history = backbone_model.fit(ds, epochs=epochs, callbacks=callbacks)
    return history

# TESTING FUNCTION

In [ ]:
# %%
# Embedding extraction & evaluation

def extract_embeddings(model, filepaths, batch_size=BATCH_SIZE):
    ds = tf.data.Dataset.from_tensor_slices(filepaths)
    ds = ds.map(lambda x: preprocess_image(x), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    embs = []
    for b in ds:
        e = model(b, training=False).numpy()
        embs.append(e)
    embs = np.vstack(embs)
    return embs


def evaluate_identification(emb_gallery, labels_gallery, emb_probe, labels_probe, topk=(1,5)):
    # compute distances (cosine via normalized embeddings -> use dot)
    sims = np.dot(emb_probe, emb_gallery.T)  # higher is better
    ranks = []
    for i in range(sims.shape[0]):
        order = np.argsort(-sims[i])  # descending
        gt = labels_probe[i]
        ranked_labels = [labels_gallery[j] for j in order]
        rk = {}
        for k in topk:
            rk[k] = 1 if gt in ranked_labels[:k] else 0
        ranks.append(rk)
    # compute metrics
    metrics = {f'Rank-{k}': np.mean([r[k] for r in ranks]) for k in topk}
    return metrics


def compute_roc_eer(emb1, labels1, emb2, labels2, n_pairs=10000):
    # sample same/diff pairs and compute distances
    same_scores = []
    diff_scores = []
    rng = np.random.RandomState(0)
    # same pairs
    for _ in range(n_pairs//2):
        cls = rng.choice(list(set(labels1)))
        idxs = [i for i,l in enumerate(labels1) if l==cls]
        if len(idxs) < 2:
            continue
        i,j = rng.choice(idxs, size=2, replace=False)
        s = np.dot(emb1[i], emb2[j])
        same_scores.append(s)
    # diff pairs
    for _ in range(n_pairs//2):
        a,b = rng.choice(len(labels1), size=2, replace=False)
        if labels1[a] == labels2[b]:
            continue
        s = np.dot(emb1[a], emb2[b])
        diff_scores.append(s)
    y = np.array([1]*len(same_scores) + [0]*len(diff_scores))
    scores = np.array(same_scores + diff_scores)
    fpr, tpr, thresholds = roc_curve(y, scores)
    roc_auc = auc(fpr, tpr)
    # EER approximate
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.absolute((fnr - fpr)))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
    return fpr, tpr, roc_auc, eer

# RUN FUNCTION

In [ ]:
# %%
# High-level experiment runner

def run_experiment(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    outdir = os.path.join(OUTPUT_DIR, f'result_seed_{seed}')
    os.makedirs(outdir, exist_ok=True)

    # 1) make splits
    gallery, probe = make_splits(DATA_DIR, gallery_k=GALLERY_K, seed=seed)
    # flatten filepaths & labels
    gallery_files = []
    gallery_labels = []
    for cls, files in gallery.items():
        gallery_files += files
        gallery_labels += [cls]*len(files)

    probe_files = []
    probe_labels = []
    for cls, files in probe.items():
        probe_files += files
        probe_labels += [cls]*len(files)

    # 2) Build models
    backbone_model, base = build_backbone()
    # Save a copy of the backbone weights for reuse
    backbone_model.save_weights(os.path.join(outdir, 'backbone_init_weights.h5'))

    # A: Frozen backbone + train head only (we will train a small head classifier as proxy)
    # Build classifier model on top of base (we must create a fresh base and load weights)
    num_classes = len(sorted(os.listdir(DATA_DIR)))
    # Create a backbone copy for classifier
    cls_backbone, cls_base = build_backbone()
    cls_backbone.load_weights(os.path.join(outdir, 'backbone_init_weights.h5'), by_name=True)

    classifier_model = build_classifier_model(num_classes, cls_backbone, freeze_backbone=True, unfreeze_last_n=0)

    # prepare train/val splits for classifier training: use gallery as train and subset of probe as val
    # NOTE: This is a pragmatic approach because you have limited images per class.
    train_files = gallery_files.copy()
    train_labels = [gallery_labels[i] for i in range(len(gallery_files))]
    val_files = []
    val_labels = []
    for cls, files in probe.items():
        # take up to 2 images per class for val
        picks = files[:2]
        val_files += picks
        val_labels += [cls]*len(picks)

    # map labels to ints
    class_list = sorted(os.listdir(DATA_DIR))
    class_to_int = {c:i for i,c in enumerate(class_list)}
    train_labels_int = [class_to_int[c] for c in train_labels]
    val_labels_int = [class_to_int[c] for c in val_labels]

    train_ds = make_dataset_from_filepaths(train_files, train_labels_int, batch_size=BATCH_SIZE, shuffle=True)
    val_ds = make_dataset_from_filepaths(val_files, val_labels_int, batch_size=BATCH_SIZE, shuffle=False)

    print('Training classifier head (Frozen backbone) ...')
    history_a = train_classifier(classifier_model, train_ds, val_ds, outdir, epochs=EPOCHS, lr=1e-3)
    classifier_model.save(os.path.join(outdir, 'classifier_model.h5'))

    # Extract embeddings for gallery & probe using classifier backbone (penultimate)
    # Build embedding extractor from base backbone weights
    emb_model_A, _ = build_backbone()
    emb_model_A.load_weights(os.path.join(outdir, 'backbone_init_weights.h5'), by_name=True)
    # The head trained above won't affect embeddings; for a fairer test you can train a small bottleneck on top of frozen backbone instead.

    emb_gallery_A = extract_embeddings(emb_model_A, gallery_files)
    emb_probe_A = extract_embeddings(emb_model_A, probe_files)

    metrics_A = evaluate_identification(emb_gallery_A, gallery_labels, emb_probe_A, probe_labels)
    print('A (frozen backbone) metrics:', metrics_A)

    # B: Finetune last block + head (BN frozen)
    print('Preparing finetune model (last block) ...')
    fin_backbone, fin_base = build_backbone()
    fin_backbone.load_weights(os.path.join(outdir, 'backbone_init_weights.h5'), by_name=True)
    # Freeze all BN
    for layer in fin_base.layers:
        layer.trainable = False
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
    # unfreeze last N layers (excluding BN)
    N = 40
    for layer in fin_base.layers[-N:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
    # add classification head
    inputs = fin_base.input
    x = fin_base.output
    logits = layers.Dense(num_classes, activation='softmax', name='Logits')(x)
    finetune_model = tf.keras.Model(inputs=inputs, outputs=logits)

    history_b = train_classifier(finetune_model, train_ds, val_ds, outdir, epochs=EPOCHS, lr=1e-5)
    finetune_model.save(os.path.join(outdir, 'finetune_model.h5'))

    # Embedding extractor for finetuned model: take penultimate from fin_backbone+head
    emb_model_B, _ = build_backbone()
    # load finetuned weights into emb_model_B by matching layer names where possible
    try:
        emb_model_B.load_weights(os.path.join(outdir, 'finetune_model.h5'), by_name=True)
    except Exception as e:
        # fallback: use initial backbone weights (explain in report)
        print('Warning: could not load finetuned weights into embedding extractor:', e)
        emb_model_B.load_weights(os.path.join(outdir, 'backbone_init_weights.h5'), by_name=True)

    emb_gallery_B = extract_embeddings(emb_model_B, gallery_files)
    emb_probe_B = extract_embeddings(emb_model_B, probe_files)
    metrics_B = evaluate_identification(emb_gallery_B, gallery_labels, emb_probe_B, probe_labels)
    print('B (finetune last block) metrics:', metrics_B)

    # C: Triplet siamese (TripletSemiHardLoss with Keras)
    print('Training triplet siamese network...')
    trip_backbone, trip_base = build_backbone()
    trip_backbone.load_weights(os.path.join(outdir, 'backbone_init_weights.h5'), by_name=True)
    # Unfreeze last N layers (keep BN frozen)
    for layer in trip_base.layers:
        layer.trainable = False
    for layer in trip_base.layers[-40:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
    # prepare filepaths and labels for Keras dataset
    all_files = []
    all_labels = []
    for cls in sorted(os.listdir(DATA_DIR)):
        cls_dir = os.path.join(DATA_DIR, cls)
        imgs = sorted([os.path.join(cls_dir, f) for f in os.listdir(cls_dir) if f.lower().endswith(('.jpg','.png'))])
        all_files += imgs
        all_labels += [cls]*len(imgs)
    # map labels to ints
    unique = sorted(list(set(all_labels)))
    label_to_int = {c:i for i,c in enumerate(unique)}
    int_labels = [label_to_int[c] for c in all_labels]
    ds_files = tf.data.Dataset.from_tensor_slices((all_files, int_labels))
    def _map(p,l): return preprocess_image(p), l
    ds_files = ds_files.map(lambda p,l: _map(p,l), num_parallel_calls=AUTOTUNE)
    ds_files = ds_files.shuffle(2048).batch(BATCH_SIZE).prefetch(AUTOTUNE)

    trip_backbone.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=tf.keras.losses.TripletSemiHardLoss())
    callbacks = [tf.keras.callbacks.ModelCheckpoint(os.path.join(outdir, 'triplet_best.h5'), save_best_only=True)]
    history_c = trip_backbone.fit(ds_files, epochs=EPOCHS, callbacks=callbacks)
    trip_backbone.save(os.path.join(outdir, 'triplet_model.h5'))

    emb_model_C, _ = build_backbone()
    try:
        emb_model_C.load_weights(os.path.join(outdir, 'triplet_model.h5'), by_name=True)
    except Exception as e:
        print('Warning loading triplet weights into extractor:', e)
        emb_model_C.load_weights(os.path.join(outdir, 'backbone_init_weights.h5'), by_name=True)

    emb_gallery_C = extract_embeddings(emb_model_C, gallery_files)
    emb_probe_C = extract_embeddings(emb_model_C, probe_files)
    metrics_C = evaluate_identification(emb_gallery_C, gallery_labels, emb_probe_C, probe_labels)
    print('C (triplet) metrics:', metrics_C)

    # Save metrics and histories
    np.savez(os.path.join(outdir, 'embeddings_and_metrics.npz'),
             gallery_files=gallery_files, gallery_labels=gallery_labels,
             probe_files=probe_files, probe_labels=probe_labels,
             emb_gallery_A=emb_gallery_A, emb_probe_A=emb_probe_A,
             emb_gallery_B=emb_gallery_B, emb_probe_B=emb_probe_B,
             emb_gallery_C=emb_gallery_C, emb_probe_C=emb_probe_C,
             metrics_A=metrics_A, metrics_B=metrics_B, metrics_C=metrics_C)

    # Plot t-SNE for each method
    def plot_tsne(embs, labels, title, fname):
        ts = TSNE(n_components=2, perplexity=30, random_state=0).fit_transform(embs)
        plt.figure(figsize=(8,8))
        palette = sns.color_palette('hls', len(set(labels)))
        lbls = [labels]
        sns.scatterplot(x=ts[:,0], y=ts[:,1], hue=labels, legend=False, palette=palette)
        plt.title(title)
        plt.savefig(fname)
        plt.close()

    plot_tsne(np.vstack([emb_gallery_A, emb_probe_A]), gallery_labels+probe_labels, 'A - Frozen', os.path.join(outdir,'tsne_A.png'))
    plot_tsne(np.vstack([emb_gallery_B, emb_probe_B]), gallery_labels+probe_labels, 'B - Finetune', os.path.join(outdir,'tsne_B.png'))
    plot_tsne(np.vstack([emb_gallery_C, emb_probe_C]), gallery_labels+probe_labels, 'C - Triplet', os.path.join(outdir,'tsne_C.png'))

    # Save training curves
    def save_history(h, name):
        try:
            plt.figure(); plt.plot(h.history['loss'], label='train'); plt.plot(h.history.get('val_loss', []), label='val'); plt.title(name); plt.legend(); plt.savefig(os.path.join(outdir, f'{name}.png')); plt.close()
        except Exception as e:
            pass
    save_history(history_a, 'history_A')
    save_history(history_b, 'history_B')
    save_history(history_c, 'history_C')

    return {'A': metrics_A, 'B': metrics_B, 'C': metrics_C}

# ORCHESTRATOR

In [ ]:
# %%
# Run for multiple seeds and collect results
all_results = {}
for s in SEEDS:
    print('\n=== Running seed', s)
    res = run_experiment(s)
    all_results[s] = res
    print('Results seed', s, res)

# Save aggregated results
import json
with open(os.path.join(OUTPUT_DIR, 'all_results.json'), 'w') as f:
    json.dump(all_results, f)

print('Done. Results saved to', OUTPUT_DIR)